In [1]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np # Importamos numpy para usar NaN

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# TODO: NOME EXATO DA PLANILHA (Usando a folha 'BASE' que é a mais limpa)
NOME_ARQUIVO = '2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv'
LINHA_CABECALHO = 0 # O cabeçalho parece estar na primeira linha (índice 0)

# Mapeamento: os nomes REAIS para os nomes do banco
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    # Lendo o CSV. Como é CSV, precisamos do separador (',')
    df = pd.read_csv(CAMINHO_BRUTO, header=LINHA_CABECALHO, sep=',')
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
except Exception as e:
    print(f"❌ ERRO NA LEITURA: {e}")
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# 1. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Remove a coluna de lixo que pode ter vindo do final do CSV
if df_limpo.columns[-1] == 'Unnamed: 7':
    df_limpo = df_limpo.drop(columns=['Unnamed: 7'])

# Remove linhas onde o valor ou a data principal estão vazios (Lixo)
df_limpo.dropna(subset=['valor', 'data_movimentacao'], inplace=True) 

# Converte o valor para numérico, forçando o tratamento de strings como 'R$ 1.000,00'
df_limpo['valor'] = df_limpo['valor'].astype(str).str.replace(r'[^\d,-]', '', regex=True).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_limpo['valor'] = pd.to_numeric(df_limpo['valor'], errors='coerce')

# Garante que a data está no formato correto
df_limpo['data_movimentacao'] = pd.to_datetime(df_limpo['data_movimentacao'], errors='coerce')
df_limpo.dropna(subset=['data_movimentacao'], inplace=True) # Remove linhas onde a data é inválida

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL
colunas_finais_sql = ['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome', 'valor', 'tipo_transacao', 'centro_custo']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
# O mesmo código de conexão que funcionou antes!
print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO ---
DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' # Sua senha
DB_HOST = '127.0.0.1'     
DB_PORT = '5432'          
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) 
NOME_TABELA_SQL = 'ft_movimentacao_financeira' # <--- NOVA TABELA

try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: 2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv
--- FASE 1: EXTRAÇÃO (READ) ---
❌ ERRO NA LEITURA: [Errno 2] No such file or directory: '..\\01_raw_data\\2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv'


FileNotFoundError: [Errno 2] No such file or directory: '..\\01_raw_data\\2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv'

In [2]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# TODO: COPIE O NOME EXATO DO ARQUIVO CSV DA SUA PASTA '01_raw_data'
# MANTENHA A EXTENSÃO .csv NO FINAL
NOME_ARQUIVO = '2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv' # NOME PROVÁVEL
LINHA_CABECALHO = 0 

# Mapeamento: os nomes REAIS para os nomes do banco
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    # Lendo o CSV. 
    df = pd.read_csv(CAMINHO_BRUTO, header=LINHA_CABECALHO, sep=',')
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
except FileNotFoundError as e:
    print(f"❌ ERRO CRÍTICO: Arquivo '{NOME_ARQUIVO}' NÃO ENCONTRADO.")
    print("   VERIFIQUE SE O ARQUIVO ESTÁ NA PASTA 01_raw_data E SE O NOME ACIMA É EXATO (incluindo o .csv).")
    raise
except Exception as e:
    print(f"❌ ERRO NA LEITURA: {e}")
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# 1. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Remove a coluna de lixo que pode ter vindo do final do CSV
if df_limpo.columns[-1] == 'Unnamed: 7':
    df_limpo = df_limpo.drop(columns=['Unnamed: 7'])

# Remove linhas onde o valor ou a data principal estão vazios (Lixo)
df_limpo.dropna(subset=['valor', 'data_movimentacao'], inplace=True) 

# Converte o valor para numérico, forçando o tratamento de strings
df_limpo['valor'] = df_limpo['valor'].astype(str).str.replace(r'[^\d,-]', '', regex=True).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_limpo['valor'] = pd.to_numeric(df_limpo['valor'], errors='coerce')

# Garante que a data está no formato correto
df_limpo['data_movimentacao'] = pd.to_datetime(df_limpo['data_movimentacao'], errors='coerce', dayfirst=True) # Adicionando dayfirst=True para formato BR
df_limpo.dropna(subset=['data_movimentacao'], inplace=True) 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL
colunas_finais_sql = ['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome', 'valor', 'tipo_transacao', 'centro_custo']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO ---
DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' # Sua senha
DB_HOST = '127.0.0.1'     
DB_PORT = '5432'          
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) 
NOME_TABELA_SQL = 'ft_movimentacao_financeira' 

try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: 2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv
--- FASE 1: EXTRAÇÃO (READ) ---
❌ ERRO CRÍTICO: Arquivo '2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv' NÃO ENCONTRADO.
   VERIFIQUE SE O ARQUIVO ESTÁ NA PASTA 01_raw_data E SE O NOME ACIMA É EXATO (incluindo o .csv).


FileNotFoundError: [Errno 2] No such file or directory: '..\\01_raw_data\\2 atualizada PLANEJAMENTO JULHO Á DEZEMBRO----.xlsx - BASE.csv'

In [3]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np 

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# NOME SIMPLIFICADO: VOCÊ DEVE RENOMEAR O ARQUIVO PARA ISSO
NOME_ARQUIVO = 'FINANCEIRO_BASE.csv'
LINHA_CABECALHO = 0 

# Mapeamento: os nomes REAIS para os nomes do banco
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    # Lendo o CSV. 
    df = pd.read_csv(CAMINHO_BRUTO, header=LINHA_CABECALHO, sep=',')
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
except FileNotFoundError as e:
    print(f"❌ ERRO CRÍTICO: Arquivo '{NOME_ARQUIVO}' NÃO ENCONTRADO.")
    print("   VERIFIQUE SE O ARQUIVO FOI RENOMEADO CORRETAMENTE PARA FINANCEIRO_BASE.csv.")
    raise
except Exception as e:
    print(f"❌ ERRO NA LEITURA: {e}")
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# 1. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Remove a coluna de lixo que pode ter vindo do final do CSV
if df_limpo.columns[-1] == 'Unnamed: 7':
    df_limpo = df_limpo.drop(columns=['Unnamed: 7'])

# Remove linhas onde o valor ou a data principal estão vazios (Lixo)
df_limpo.dropna(subset=['valor', 'data_movimentacao'], inplace=True) 

# Converte o valor para numérico, forçando o tratamento de strings
df_limpo['valor'] = df_limpo['valor'].astype(str).str.replace(r'[^\d,-]', '', regex=True).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_limpo['valor'] = pd.to_numeric(df_limpo['valor'], errors='coerce')

# Garante que a data está no formato correto
df_limpo['data_movimentacao'] = pd.to_datetime(df_limpo['data_movimentacao'], errors='coerce', dayfirst=True) # Adicionando dayfirst=True para formato BR
df_limpo.dropna(subset=['data_movimentacao'], inplace=True) 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL
colunas_finais_sql = ['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome', 'valor', 'tipo_transacao', 'centro_custo']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO ---
DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' # Sua senha
DB_HOST = '127.0.0.1'     
DB_PORT = '5432'          
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) 
NOME_TABELA_SQL = 'ft_movimentacao_financeira' 

try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: FINANCEIRO_BASE.csv
--- FASE 1: EXTRAÇÃO (READ) ---
❌ ERRO CRÍTICO: Arquivo 'FINANCEIRO_BASE.csv' NÃO ENCONTRADO.
   VERIFIQUE SE O ARQUIVO FOI RENOMEADO CORRETAMENTE PARA FINANCEIRO_BASE.csv.


FileNotFoundError: [Errno 2] No such file or directory: '..\\01_raw_data\\FINANCEIRO_BASE.csv'

In [4]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np 

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# CORREÇÃO FINAL: Nome do arquivo renomeado e com extensão .xlsx
NOME_ARQUIVO = 'FINANCEIRO_BASE.xlsx' 
LINHA_CABECALHO = 0 # O cabeçalho está na primeira linha (índice 0)

# Mapeamento: os nomes REAIS para os nomes do banco (Estes nomes estão corretos)
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    # USANDO read_excel para ler o .xlsx
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
except FileNotFoundError as e:
    print(f"❌ ERRO CRÍTICO: Arquivo '{NOME_ARQUIVO}' NÃO ENCONTRADO.")
    print("   VERIFIQUE SE O ARQUIVO ESTÁ NA PASTA 01_raw_data E SE O NOME É EXATO (FINANCEIRO_BASE.xlsx).")
    raise
except Exception as e:
    print(f"❌ ERRO NA LEITURA: {e}")
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# 1. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Selecionando as colunas mapeadas
df_limpo = df_limpo[colunas_finais_mapeadas] 

# Remove linhas onde o valor ou a data principal estão vazios (Lixo)
df_limpo.dropna(subset=['valor', 'data_movimentacao'], inplace=True) 

# Converte o valor para numérico, forçando o tratamento de strings
df_limpo['valor'] = df_limpo['valor'].astype(str).str.replace(r'[^\d,-]', '', regex=True).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_limpo['valor'] = pd.to_numeric(df_limpo['valor'], errors='coerce')

# Garante que a data está no formato correto
df_limpo['data_movimentacao'] = pd.to_datetime(df_limpo['data_movimentacao'], errors='coerce', dayfirst=True) 
df_limpo.dropna(subset=['data_movimentacao'], inplace=True) 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL
colunas_finais_sql = ['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome', 'valor', 'tipo_transacao', 'centro_custo']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO ---
DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' # Sua senha
DB_HOST = '127.0.0.1'     
DB_PORT = '5432'          
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) 
NOME_TABELA_SQL = 'ft_movimentacao_financeira' 

try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: FINANCEIRO_BASE.xlsx
--- FASE 1: EXTRAÇÃO (READ) ---
✅ Arquivo lido com sucesso. Dimensões brutas: (4, 7)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---


KeyError: "None of [Index(['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome',\n       'valor', 'tipo_transacao', 'centro_custo'],\n      dtype='object')] are in the [columns]"

In [5]:
# ... (O código da FASE 1 é o mesmo)

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# --- NOVO CÓDIGO DE LIMPEZA DE CABEÇALHOS (CORREÇÃO DA KEYERROR) ---

# Limpa: Remove espaços extras (strip) e padroniza para MAIÚSCULAS (upper)
df.columns = df.columns.str.strip().str.upper()

# Mapeamento: os nomes REAIS para os nomes do banco
# Os nomes à esquerda DEVEM estar em MAIÚSCULAS para bater com a linha acima
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

# 1. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Selecionando as colunas mapeadas
df_limpo = df_limpo[colunas_finais_mapeadas] 

# ... (O restante do código de transformação é o mesmo)


--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---


KeyError: "None of [Index(['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome',\n       'valor', 'tipo_transacao', 'centro_custo'],\n      dtype='object')] are in the [columns]"

In [6]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np 

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# CORREÇÃO FINAL: Nome do arquivo renomeado e com extensão .xlsx
NOME_ARQUIVO = 'FINANCEIRO_BASE.xlsx' 
LINHA_CABECALHO = 0 

# Mapeamento: os nomes REAIS para os nomes do banco
# (Vamos assumir MAIÚSCULAS para a esquerda, mas vamos inspecionar)
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

# --- FASE 1: EXTRAÇÃO (READ) ---
try:
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
except Exception as e:
    raise

print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# --- NOVO CÓDIGO DE LIMPEZA E DEBUG ---

# 1. Limpa: Remove espaços extras (strip) e padroniza para MAIÚSCULAS (upper)
df.columns = df.columns.astype(str).str.strip().str.upper()

# 2. DEBUG: MOSTRA O NOME FINAL DA COLUNA NO DATAFRAME
print("--- CABEÇALHOS DO DATAFRAME APÓS A LIMPEZA ---")
print(list(df.columns)) 
print("---------------------------------------------")

# AGORA REPETIMOS O ERRO DE PROPÓSITO, MAS TEREMOS A INFORMAÇÃO ANTES DE FALHAR

# 3. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)

# O código vai falhar aqui de novo, mas teremos a informação que precisamos!
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())
df_limpo = df_limpo[colunas_finais_mapeadas] 

# [O restante do código da FASE 2 e FASE 3 é o mesmo e está funcional, mas não será executado]

✅ Arquivo lido com sucesso. Dimensões brutas: (4, 7)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---
--- CABEÇALHOS DO DATAFRAME APÓS A LIMPEZA ---
['DETALHES DO SOMA DE VALOR - TIPO: RECEITAS, DIAS (DATA): 15-SEP', 'UNNAMED: 1', 'UNNAMED: 2', 'UNNAMED: 3', 'UNNAMED: 4', 'UNNAMED: 5', 'UNNAMED: 6']
---------------------------------------------


KeyError: "None of [Index(['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome',\n       'valor', 'tipo_transacao', 'centro_custo'],\n      dtype='object')] are in the [columns]"

In [7]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np 

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# NOME FINAL: FINANCEIRO_BASE.xlsx
NOME_ARQUIVO = 'FINANCEIRO_BASE.xlsx' 
LINHA_CABECALHO = 1 # <--- MUDANÇA: Tentamos o índice 1 (Segunda linha do Excel)

# Mapeamento: os nomes REAIS para os nomes do banco 
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    # MUDANÇA: Usando header=LINHA_CABECALHO (agora é 1)
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
except Exception as e:
    raise

print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# --- CORREÇÃO DE CABEÇALHO ---
# 1. Limpa: Remove espaços extras (strip) e padroniza para MAIÚSCULAS (upper)
# Isso deve ser mantido, pois resolve espaços invisíveis.
df.columns = df.columns.astype(str).str.strip().str.upper()

# 2. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Selecionando as colunas mapeadas
df_limpo = df_limpo[colunas_finais_mapeadas] 

# [O restante do código da FASE 2 e FASE 3 é o mesmo]

# Remove linhas onde o valor ou a data principal estão vazios (Lixo)
df_limpo.dropna(subset=['valor', 'data_movimentacao'], inplace=True) 

# Converte o valor para numérico, forçando o tratamento de strings
df_limpo['valor'] = df_limpo['valor'].astype(str).str.replace(r'[^\d,-]', '', regex=True).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_limpo['valor'] = pd.to_numeric(df_limpo['valor'], errors='coerce')

# Garante que a data está no formato correto
df_limpo['data_movimentacao'] = pd.to_datetime(df_limpo['data_movimentacao'], errors='coerce', dayfirst=True) 
df_limpo.dropna(subset=['data_movimentacao'], inplace=True) 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL
colunas_finais_sql = ['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome', 'valor', 'tipo_transacao', 'centro_custo']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
# ... (O restante do código da Carga é o mesmo e está funcional)
DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' 
DB_HOST = '127.0.0.1'     
DB_PORT = '5432'          
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) 
NOME_TABELA_SQL = 'ft_movimentacao_financeira' 

try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: FINANCEIRO_BASE.xlsx
--- FASE 1: EXTRAÇÃO (READ) ---
✅ Arquivo lido com sucesso. Dimensões brutas: (3, 7)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---


KeyError: "None of [Index(['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome',\n       'valor', 'tipo_transacao', 'centro_custo'],\n      dtype='object')] are in the [columns]"

In [8]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np 

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---
# ----------------------------------------------------

# NOME FINAL: FINANCEIRO_BASE.xlsx
NOME_ARQUIVO = 'FINANCEIRO_BASE.xlsx' 
LINHA_CABECALHO = 2 # <--- CORREÇÃO FINAL: Tentando o índice 2 (Quarta linha do Excel)

# Mapeamento: os nomes REAIS para os nomes do banco 
COLUNAS_PARA_MANTER = {
    'DATA': 'data_movimentacao',
    'MES': 'mes',
    'EMPRESA': 'unidade_negocio_nome',
    'FORNECEDOR': 'contraparte_nome',
    'VALOR': 'valor',
    'TIPO': 'tipo_transacao',
    'CENTRO DE CUSTO': 'centro_custo'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    # MUDANÇA: Usando header=LINHA_CABECALHO (agora é 2)
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
except Exception as e:
    print(f"❌ ERRO NA LEITURA: {e}")
    raise

print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# 1. Limpa e Padroniza Cabeçalhos: Remove espaços e padroniza para MAIÚSCULAS
df.columns = df.columns.astype(str).str.strip().str.upper()

# 2. Renomeação, Seleção e Limpeza Básica
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Selecionando as colunas mapeadas
df_limpo = df_limpo[colunas_finais_mapeadas] 

# Remove linhas onde o valor ou a data principal estão vazios (Lixo)
df_limpo.dropna(subset=['valor', 'data_movimentacao'], inplace=True) 

# Converte o valor para numérico, forçando o tratamento de strings
df_limpo['valor'] = df_limpo['valor'].astype(str).str.replace(r'[^\d,-]', '', regex=True).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_limpo['valor'] = pd.to_numeric(df_limpo['valor'], errors='coerce')

# Garante que a data está no formato correto
df_limpo['data_movimentacao'] = pd.to_datetime(df_limpo['data_movimentacao'], errors='coerce', dayfirst=True) 
df_limpo.dropna(subset=['data_movimentacao'], inplace=True) 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL
colunas_finais_sql = ['data_movimentacao', 'mes', 'unidade_negocio_nome', 'contraparte_nome', 'valor', 'tipo_transacao', 'centro_custo']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
# ... (O restante do código de Carga é o mesmo e está funcional)

DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' 
DB_HOST = '127.0.0.1'     
DB_PORT = '5432'          
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) 
NOME_TABELA_SQL = 'ft_movimentacao_financeira' 

try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: FINANCEIRO_BASE.xlsx
--- FASE 1: EXTRAÇÃO (READ) ---
✅ Arquivo lido com sucesso. Dimensões brutas: (2, 7)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---
✅ Mapeamento, limpeza e padronização concluídos.
Dimensões após a limpeza: (2, 7)
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 2 linhas na tabela 'ft_movimentacao_financeira'...
✅ SUCESSO! Dados carregados na tabela 'ft_movimentacao_financeira'.
